In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import panel as pn
import time
import random
import copy
import rtsvg
rt = rtsvg.RACETrack()

df1 = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk1.csv')
#df2 = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk2.csv')
#df3 = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk3.csv')
df  = pl.concat([df1]) # , df2, df3])

df = rt.columnsAreTimestamps(df, 'parsedDate')
df = df.rename({'TimeSeconds':                '_del1_',    'parsedDate':                 'timestamp',     'dateTimeStr':                '_del2_',
                'ipLayerProtocol':            'pro',       'ipLayerProtocolCode':        '_del3_',        'firstSeenSrcIp':             'sip',
                'firstSeenDestIp':            'dip',       'firstSeenSrcPort':           'spt',           'firstSeenDestPort':          'dpt',
                'moreFragments':              '_del4_',    'contFragments':              '_del5_',        'durationSeconds':            'dur',
                'firstSeenSrcPayloadBytes':   '_del6_',    'firstSeenDestPayloadBytes':  '_del7_',        'firstSeenSrcTotalBytes':     'soct',
                'firstSeenDestTotalBytes':    'doct',      'firstSeenSrcPacketCount':    'spkt',          'firstSeenDestPacketCount':   'dpkt',
                'recordForceOut':             '_del8_'})
df = df.drop(['_del1_', '_del2_', '_del3_', '_del4_', '_del5_', '_del6_', '_del7_', '_del8_'])
df = df.sample(1_000)
_relates_ = [('sip','dip')]
#params_rt = {'df':df, 'relationships':_relates_, 'node_color':'node', 'every':'1d', 'h':384}
#sl        = rt.spreadLines(node_focus='172.10.0.6', **params_rt)
#rt.tile([sl])

In [ ]:
_spec_    = {
             (0,0,1,1):('linkNode',    {'relationships':_relates_}),
             (1,0,1,1):('histogram',   {'bin_by':'sip'}),
             (2,0,4,1):('spreadLines', {'relationships':_relates_, 'node_focus':'172.10.0.6'})
            }
_ip_      = rt.interactivePanel(df, _spec_, 1000, 200)
_ip_

In [ ]:
_ln_params_ = {'relationships':_relates_, 'pos':{}, 'link_shape':'curve', 'link_size':'vary'}
_igl_       = rt.interactiveGraphPanel(df, _ln_params_, w=600, h=600)
_ln_params_ = {'relationships':_relates_, 'pos':{}, 'link_shape':'curve', 'link_size':'vary'}
_igl2_      = rt.interactiveGraphPanel(df, _ln_params_, w=600, h=600)
_histogram_ = rt.interactivePanel(df, {(0,0,1,1):('histogram', {'bin_by':'sip', 'count_by':'dip'})}, 200, 600)
_igl_.register_companion_viz(_histogram_),     _igl_.register_companion_viz(_igl2_)
_igl2_.register_companion_viz(_histogram_),    _igl2_.register_companion_viz(_igl_)
_histogram_.register_companion_viz(_igl_),     _histogram_.register_companion_viz(_igl2_)
pn.Row(_igl_, _igl2_, _histogram_) #.show()